# Process original data

In [63]:
import pandas as pd

#Read "CCDS.20160908_origin.txt" as csv file:

In [64]:
data0 = pd.read_csv("CCDS.20160908_origin.txt",sep="\t")

In [65]:
data0.shape

(34297, 11)

#This data frame contains 34297 rows and 11 columns

#Check the firs 5 rows of data0:

In [66]:
data0.head()

,#chromosome,nc_accession,gene,gene_id,ccds_id,ccds_status,cds_strand,cds_from,cds_to,cds_locations,match_type
0,1,NC_000001.8,LINC00115,79854,CCDS1.1,Withdrawn,-,-,-,-,None
1,1,NC_000001.11,SAMD11,148398,CCDS2.2,Public,+,925941,944152,"[925941-926012, 930154-930335, 931038-931088, ...",Identical
2,1,NC_000001.11,NOC2L,26155,CCDS3.1,Public,-,944693,959239,"[944693-944799, 945056-945145, 945517-945652, ...",Identical
3,1,NC_000001.11,PLEKHN1,84069,CCDS4.1,Public,+,966531,974574,"[966531-966613, 966703-966802, 970276-970422, ...",Identical
4,1,NC_000001.11,HES4,57801,CCDS5.1,Public,-,999058,999972,"[999058-999431, 999525-999612, 999691-999786, ...",Identical


#Now I can see that the column labels including: "#chromosome", "nc_accession", "gene", "gene_id", "ccds_id",
"ccds_status", "cds_strand", "cds_from", "cds_to", "cds_locations", "match_type"

#Check the composition of the "match_type" column of data0:

In [67]:
data0.iloc[:,-1].value_counts()

Identical    34031
None           219
Partial         47
Name: match_type, dtype: int64

#I only want to take the data whose "match_type" is "Identical"

#Check the composition of the "ccds_status" column of data0:

In [68]:
data0.iloc[:,5].value_counts()

Public                                32534
Withdrawn                              1731
Under review, withdrawal                 15
Reviewed, update pending                  7
Withdrawn, inconsistent annotation        6
Under review, update                      4
Name: ccds_status, dtype: int64

#I only want to take the data whose "ccds_status" is "Public"

In [69]:
import numpy as np

#Take the data with "match_type"=="Identical" and "ccds_status"=="Public", and save as data1:

In [70]:
data1 = data0[np.logical_and(data0.iloc[:,5]=="Public",data0.iloc[:,-1]=="Identical")]

Because the "match_type" of all chosen genes is "Identical" and the "ccds_status" of all chosen genes is "Public", I can drop the "match_type" column and "ccds_status" column:

In [71]:
cds = data1.drop(["match_type", "ccds_status"],axis=1)

In [72]:
cds.head()

,#chromosome,nc_accession,gene,gene_id,ccds_id,cds_strand,cds_from,cds_to,cds_locations
1,1,NC_000001.11,SAMD11,148398,CCDS2.2,+,925941,944152,"[925941-926012, 930154-930335, 931038-931088, ..."
2,1,NC_000001.11,NOC2L,26155,CCDS3.1,-,944693,959239,"[944693-944799, 945056-945145, 945517-945652, ..."
3,1,NC_000001.11,PLEKHN1,84069,CCDS4.1,+,966531,974574,"[966531-966613, 966703-966802, 970276-970422, ..."
4,1,NC_000001.11,HES4,57801,CCDS5.1,-,999058,999972,"[999058-999431, 999525-999612, 999691-999786, ..."
5,1,NC_000001.11,ISG15,9636,CCDS6.1,+,1013573,1014477,"[1013573-1013575, 1013983-1014477]"


# Calculate the total length of CDS (coding sequences) for all genes

#I want to use the information in the "cds_locations" column to calculate the length of CDS.
For example, for the cds_location "925941-926012", the CDS length = 926012-925941+1

#But, before I do the calculation, I need to further process the data:

#For example, the first gene:

In [73]:
tmp = cds.iloc[0,-1]

In [74]:
print(tmp)

[925941-926012, 930154-930335, 931038-931088, 935771-935895, 939039-939128, 939274-939459, 941143-941305, 942135-942250, 942409-942487, 942558-943057, 943252-943376, 943697-943807, 943907-944152]


In [75]:
type(tmp)

str

#I can not do the calculation with strings, so I need to change the data type from strings to integers. I also need to use np.diff()to calculate the length of CDS

In [76]:
tmp1 = tmp.strip("[").strip("]").split(", ")

In [77]:
tmp2 = tmp1[0]

In [78]:
print(tmp2)

925941-926012


In [79]:
tmp3 = list(map(int,tmp2.split("-")))

In [80]:
print(tmp3)

[925941, 926012]


In [81]:
cds_len1 = np.diff(list(map(int,tmp2.split("-"))))[0] + 1

In [82]:
print(cds_len1)

72


926012-925941+1=72. it works!#

#Now calculate the total CDS length for the first gene:

In [83]:
def sum_exon(tmp):
    """
    Calculate total CDS length of a gene
    
    """
    tmp1 = tmp.strip("[").strip("]").split(", ")
    cds_len = sum([np.diff(list(map(int,tmp1[i].split("-"))))[0]+1 
    for i in range(len(tmp1))])
    return(cds_len)

In [84]:
sum_exon(tmp)

2046

#Apply the sum_exon(tmp) to all the genes:

In [85]:
cds_len_list = list(map(sum_exon,cds.cds_locations))

In [86]:
print(cds_len_list[0:30])

[2046, 2250, 1836, 666, 498, 597, 1215, 768, 726, 834, 1047, 990, 780, 828, 624, 2505, 912, 1803, 2013, 1329, 600, 450, 1338, 195, 1947, 1905, 585, 801, 1023, 630]


#Calculate the total length of CDS of all the genes:

In [87]:
total_cds_len = sum(cds_len_list)

In [88]:
print("Total length of human CDS is: ",total_cds_len)

Total length of human CDS is:  55591157
